### Use the menu entry "Run &rarr; Run All Cells" to start this notebook.
This interactive Jupyter notebook is supported by <a href="https://www.mozilla.org/firefox/new/" target="_blank">Firefox</a> and
<a href="https://en.wikipedia.org/wiki/Chromium_(web_browser)#Browsers_based_on_Chromium" target="_blank">Chromium-based browsers</a> like Chrome, Opera, Vivaldi, Brave, Edge and others.

In [ ]:
# main program code

# phase 1: install all necessary packages
import gettext
import piplite
from IPython.display import clear_output, display, HTML, Javascript

gettext.bindtextdomain('kuba', 'translations')
gettext.textdomain('kuba')
_ = gettext.gettext

packages = [
    'babel',
    'folium',
    'geopandas',
    'itables',
    'ipywidgets==8.1.2',
    'ipyleaflet',
    'mapclassify',
    'openpyxl',
    'pandas'
]

progress_bar_template = """
<div style="border: 1px solid #ddd; border-radius: 4px; padding: 3px; width: 100%; margin: 10px 0;">
    <div id="progress-bar" style="background: #4caf50; width: 0%; height: 20px; border-radius: 2px;"></div>
</div>
<p id="package-name" style="text-align: center; font-weight: bold;"></p>
"""

display(HTML(progress_bar_template))

installingText = _('Installing')

def update_progress(progress, package):
    js_code = f"""
        <script>
            var bar = document.getElementById("progress-bar");
            bar.style.width = {progress} + "%";
            var packageName = document.getElementById("package-name");
            packageName.innerText = "{installingText}: " + "{package}";
        </script>
    """
    display(HTML(js_code))

total_packages = len(packages)
for i, package in enumerate(packages):
    progress = ((i + 1) / total_packages) * 100
    update_progress(progress, package)
    await piplite.install(package)

# phase 2: all necessary packages are installed
# (start the real program)
import ipywidgets as widgets

try:
    from KUBA import KUBA

    kuba = KUBA()

    def loadButtonClicked(b):
        kuba.loadBridges()

    def toggleMarkersLayer(b):
        kuba.toggleMarkerLayers()

    def updateReadout(b):
        kuba.updateReadout()

    kuba.loadButton.on_click(loadButtonClicked)
    kuba.clusterButton.observe(toggleMarkersLayer, names="value")
    kuba.bridgesSlider.observe(updateReadout, names="value")
    
except ModuleNotFoundError:
    display(widgets.HTML(_(
                """
                <h1>Startup failed</h1>
                The startup of this notebook has failed. A known cause for this
                error is starting the notebook in Firefox in private mode.
                Please try again in a new Firefox window in normal mode. More
                background information about this problem can be found here:
                <br>
                <a href="https://jupyterlite.readthedocs.io/en/latest/howto/configure/advanced/service-worker.html" target="_blank">
                https://jupyterlite.readthedocs.io/en/latest/howto/configure/advanced/service-worker.html</a>
                """
            )))